# Bouncing Ball with Manim

Run the cells below to render a bouncing ball animation using Manim inside Jupyter.


In [ ]:
%load_ext manim.utils.ipython_magic

In [ ]:
%%manim -qm -v WARNING BouncingBallWithGraph
from manim import *
import numpy as np

class BouncingBallWithGraph(Scene):
    def construct(self):
        floor_y = -2.7
        total_time = 10.0
        gravity = 9.8

        coefficient = ValueTracker(0.65)

        ground = Line(LEFT * 2.5, RIGHT * 0.5).shift(DOWN * 2.7 + LEFT * 3.5)
        restitution_label = VGroup(
            Text("Restitution").scale(0.45),
            DecimalNumber(num_decimal_places=2),
        ).arrange(RIGHT, buff=0.2).next_to(ground, UP, buff=0.2).shift(LEFT * 1.2)
        restitution_label[1].add_updater(lambda d: d.set_value(coefficient.get_value()))

        axes = Axes(
            x_range=[0, total_time, 2],
            y_range=[floor_y, 3.5, 1],
            x_length=5,
            y_length=4,
            axis_config={"include_ticks": False},
        ).to_corner(UR).shift(LEFT * 0.5)
        axes_labels = axes.get_axis_labels(Tex("t (s)"), Tex("height"))

        time_tracker = ValueTracker(0.0)
        velocity_tracker = ValueTracker(0.0)

        ball = Circle(radius=0.3, color=BLUE).set_fill(BLUE, opacity=0.9)
        ball.move_to(np.array([-3.5, 3.0, 0]))

        shadow = Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
        def update_shadow(mob):
            height = ball.get_y()
            scale = np.interp(height, [floor_y, floor_y + 3], [1.4, 0.75])
            mob.become(
                Circle(radius=0.22, color=GRAY_E, fill_opacity=0.4)
                .move_to(np.array([-3.5, floor_y + 0.02, 0]))
                .scale(scale)
            )

        shadow.add_updater(update_shadow)

        height_history = [(0.0, ball.get_y())]

        def update_ball(mob, dt):
            vy = velocity_tracker.get_value() - gravity * dt
            new_y = mob.get_y() + vy * dt

            if new_y - mob.radius <= floor_y and vy < 0:
                vy = -vy * coefficient.get_value()
                new_y = floor_y + mob.radius

            mob.move_to(np.array([mob.get_x(), new_y, 0]))
            velocity_tracker.set_value(vy)
            time_tracker.increment_value(dt)
            height_history.append((time_tracker.get_value(), new_y))

            if abs(vy) < 0.5 and abs(new_y - (floor_y + mob.radius)) < 1e-3:
                mob.remove_updater(update_ball)

        ball.add_updater(update_ball)

        velocity_arrow = always_redraw(
            lambda: Arrow(
                start=ball.get_center(),
                end=ball.get_center() + UP * velocity_tracker.get_value() * 0.12,
                buff=0,
                color=YELLOW,
                max_tip_length_to_length_ratio=0.15,
                stroke_width=5,
            )
        )

        graph = always_redraw(
            lambda: axes.plot_line_graph(
                [t for t, _ in height_history] if len(height_history) > 1 else [0, 0.0001],
                [y for _, y in height_history] if len(height_history) > 1 else [ball.get_y(), ball.get_y()],
                line_color=BLUE,
                add_vertex_dots=False,
            )
        )
        graph_dot = always_redraw(
            lambda: Dot(
                axes.c2p(time_tracker.get_value(), ball.get_y()),
                color=YELLOW,
                radius=0.06,
            )
        )

        self.play(Create(ground), FadeIn(ball), FadeIn(shadow), Create(axes), FadeIn(axes_labels), FadeIn(restitution_label))
        self.add(graph, graph_dot, velocity_arrow)

        self.wait(
            total_time,
            stop_condition=lambda: abs(velocity_tracker.get_value()) < 0.5
        )

        self.play(
            FadeOut(ball),
            FadeOut(shadow),
            FadeOut(ground),
            FadeOut(axes),
            FadeOut(axes_labels),
            FadeOut(graph),
            FadeOut(graph_dot),
            FadeOut(restitution_label),
            FadeOut(velocity_arrow),
        )
        self.wait(0.2)
